Selecionar o Caminho: C:\Users\hfeli\OneDrive\Documents\Cursos\Data Engineering\Projects\data-engineering-zoomcamp\week_1_basics_n_setup\2_docker_sql

Botão direito na pasta 2_docker_sql - Selecionar GIT Bash Here

Ligar o Docker e aguardar carregamento

No Git Bash aberto, rodar o Postgres:

winpty docker run -it \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v "c:/Users/hfeli/OneDrive/Documents/Cursos/Data Engineering/Projects/data-engineering-zoomcamp/week_1_basics_n_setup/2_docker_sql/ny_taxi_postgres_data:/var/lib/postgresql/data" \
  -p 5432:5432 \
  postgres:13

Com a mensagem: 2023-01-22 18:38:05.484 UTC [1] LOG:  database system is ready to accept connections

Abrir uma nova janela do Git Bash e rodar:

winpty pgcli -h localhost -p 5432 -u root -d ny_taxi
Em password escrever root

Abrir uma nova janela do Git Bash e instalar jupyter
pip install jupyter

Após instalado, passar o comando:
jupyter notebook

Uma nova aba com o Jupyter irá carregar no chrome, clique em *New* e selecione *Python 3*
Abrir uma nova janela do Git Bash e fazer download dos dados:
Comando: python -m wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz

Ao rodar o arquivo "upload-data.ipynb", retornar ao terminal do winpty pgcli e rodar o código:
SELECT count(1) FROM yellow_taxi_data;
Irá retornar as 1369765 linhas do arquivo original

Abrir um novo Terminal e rodar o código abaixo para conectar o PgAdmin:
winpty docker run -it \
    -e PGADMIN_DEFAULT_EMAIL="admin@admin.com" \
    -e PGADMIN_DEFAULT_PASSWORD="root" \
    -p 8080:80 \
    dpage/pgadmin4

Acessar o site localhost:8080
Colocar e-mail e senha acima
Clique no botão Add New Server
Em General -> Name -> Local docker
Connection:
host - localhost
Port 5432
Database - Postgres
Username - root
Passwrod - root

Não conecta pois estão em containers diferentes no Docker.
Criar uma conexão entre os dois para que possam se conectar

Apertar CTRL+C no terminal com Postgres para cancelar o servidor e então rodar: *docker network create pg-network* para criar a conexão
Com a conexão criada, rodar o novo código:
winpty docker run -it \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v "c:/Users/hfeli/OneDrive/Documents/Cursos/Data Engineering/Projects/data-engineering-zoomcamp/week_1_basics_n_setup/2_docker_sql/ny_taxi_postgres_data:/var/lib/postgresql/data" \
  -p 5432:5432 \
  --network=pg-network \
  --name pg-database \
  postgres:13

Rodar o PgAdmin na mesma rede:
winpty docker run -it \
    -e PGADMIN_DEFAULT_EMAIL="admin@admin.com" \
    -e PGADMIN_DEFAULT_PASSWORD="root" \
    -p 8080:80 \
    --network=pg-network \
    --name pgadmin-2 \
    dpage/pgadmin4

Abrir novamente o site localhost:8080 com o e-mail admin e senha
Criar um novo servidor e em Connection adicionar o Hostname/address *pg-database* para ele conseguir se conectar ao container através da conexão criada

Fazer um Drop Table para limpar os dados e então baixar os arquivos via código

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz"

python ingest-data.py \
    --user=root \ 
    --password=root \
    --host=localhost \
    --port=5432 \
    --db=ny_taxi \
    --table_name=yellow_taxi_trips \
    --url=${URL}

Rodar para baixar os arquivos e fazer a ingestão na base de dados via código

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz"

python ingest-data.py --user=root --password=root --host=localhost --port=5432 --db=ny_taxi --table_name=yellow_taxi_trips --url=${URL}

Criar um container do que foi criado
winpty docker build -t taxi_ingest:v001 .

Após completar, rodar o código abaixo para rodar o que está no container:

URL="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz"

winpty docker run -it \
  --network=pg-network \
  taxi_ingest:v001 \
    --user=root \
    --password=root \
    --host=pg-database \
    --port=5432 \
    --db=ny_taxi \
    --table_name=yellow_taxi_trips \
    --url=${URL}

Rodando via Localhost através de um novo terminal python -m http.server e descobrindo o IPV4: 172.20.32.1
Acessamos o 172.20.32.1:8000 e pegamos o link da planilha para baixar mais rápido

URL="http://172.20.32.1:8000/yellow_tripdata_2021-01.csv.gz"

winpty docker run -it \
  --network=pg-network \
  taxi_ingest:v001 \
    --user=root \
    --password=root \
    --host=pg-database \
    --port=5432 \
    --db=ny_taxi \
    --table_name=yellow_taxi_trips \
    --url=${URL}

Adicionado nova base de dados através do arquivo upload-data utilizando wget
Subiu automaticamente para ser visto no Pgadmin. Nova base se chama "zone"

Agora, para rodar o Localhost:8080, abrir o Git Bash e executar:
docker-compose up
Assim irá executar tanto o Postgres quando o PgAdmin ao mesmo tempo.
Quando finalizar, apertar Ctrl+C e digitar
docker-compose down
Assim irá fechar os containers

SQLs para rodar algumas queryes:
Versão 1 - sem Join
SELECT 
	tpep_pickup_datetime,
	tpep_dropoff_datetime,
	total_amount,
	CONCAT(zpu."Borough", ' \ ', zpu."Zone") AS "pick_up_loc",
	CONCAT(zdo."Borough", ' \ ', zdo."Zone") AS "dropoff_loc"
FROM 
	yellow_taxi_trips t,
	zones zpu,
	zones zdo
WHERE
	t."PULocationID" = zpu."LocationID" AND
	t."DOLocationID" = zdo."LocationID"
LIMIT 100;

Versão 2 - Com Join
SELECT 
	tpep_pickup_datetime,
	tpep_dropoff_datetime,
	total_amount,
	CONCAT(zpu."Borough", ' \ ', zpu."Zone") AS "pick_up_loc",
	CONCAT(zdo."Borough", ' \ ', zdo."Zone") AS "dropoff_loc"
FROM 
	yellow_taxi_trips t JOIN zones zpu ON t."PULocationID" = zpu."LocationID"
	JOIN zones zdo ON t."DOLocationID" = zdo."LocationID"
LIMIT 100;

Com Group By e Order By:
SELECT 
	--DATE_TRUNC('DAY', tpep_dropoff_datetime),
	CAST(tpep_dropoff_datetime AS DATE) as "day",
	"DOLocationID",
	COUNT(1) as "count",
	MAX(total_amount),
	MAX(passenger_count)
FROM 
	yellow_taxi_trips t
GROUP BY
	-- CAST(tpep_dropoff_datetime AS DATE)
	1, 2
ORDER BY "day" ASC, "DOLocationID" ASC;